In [205]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.keys import Keys
import os
from dotenv import load_dotenv
load_dotenv()
import time

In [206]:
options  = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [207]:
url = 'https://sede.madrid.es/portal/site/tramites/menuitem.62876cb64654a55e2dbd7003a8a409a0/?vgnextoid=e3b0234be7924710VgnVCM2000001f4a900aRCRD&vgnextchannel=b59637c190180210VgnVCM100000c90da8c0RCRD&vgnextfmt=pd'
driver.get(url)

In [208]:
rechazar = driver.find_element(By.XPATH, '//*[@id="iam-cookie-control-modal-action-secondary"]')
rechazar.click()

In [209]:
en_linea = driver.find_element(By.XPATH, '//*[@id="verTodas"]/div/div[1]/p/a')
en_linea.click()

In [210]:
driver.close()
driver.switch_to.window(driver.window_handles[0])

usuario_contrasena = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[1]/div[2]/div/div/div[2]/article[1]')
usuario_contrasena.click()

In [211]:
acceso = driver.find_element(By.XPATH, '//*[@id="acceso_pass"]')
acceso.click()

In [212]:
usuario = driver.find_element(By.XPATH, '//*[@id="correoelectronico"]')
usuario.send_keys(os.environ.get('user'))

contrasena = driver.find_element(By.XPATH, '//*[@id="contrasenia"]')
contrasena.send_keys(os.environ.get('pass'))
contrasena.submit()

In [213]:
actividades_dia_centro = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uSecciones_divSections"]/section[3]/div[2]/div/div/div[2]/article[1]/div')
actividades_dia_centro.click()

In [214]:
disponibilidad = driver.find_element(By.XPATH, '//*[@id="ContentFixedSection_uAltaEventos_uCentrosSeleccionar_availability_filter_on"]')
disponibilidad.click()

In [215]:
time.sleep(2)
centros = driver.find_elements(By.XPATH, "//li[contains(@class, 'media pull-left')]")

In [216]:
centros[0].click()

In [217]:
actividades = driver.find_elements(By.XPATH, '//div[contains(@class, "collapse in")]')

# Pruebas

In [225]:
actividades.element[0]

AttributeError: 'list' object has no attribute 'element'

In [ ]:
for i, centro in range(len(centros)):
    centro[i].click()
    for actividad in actividades:
        

In [186]:
print(actividades[2].text)

IndexError: list index out of range

In [218]:
len(actividades)

1

In [223]:
for element in actividades:
    texto = element.text
    print(texto)

Cubo Gym Cross Training/Funcional
Las Tablas
15 a 64 años
60'
C/ Frómista esquina C/ Castiello de Jaca
Actividad dirigida a la mejora de la condición física con acento en el trabajo de fuerza. Basa su metodología en los patrones básicos del movimiento del cuerpo, con el objetivo de mejorar la eficacia de las tareas cotidianas, profesionales, de ocio, recreativas y deportivas, en base a las necesidades individuales. Si las condiciones climatológicas son adversas, la sesión quedará suspendida.
18:00
20/20
19:00
20/20
20:00
20/20
21:00
20/20


In [135]:
for element in centros:
    texto = element.text
    print(texto)
    print('----------')

Cubo Gym Las Tablas
Calle Fromista Esq Calle Castiello de Jaca, 28050, Madrid
915880432
DISTRITO FUENCARRAL - EL PARDO
----------
Cubo Gym Madrid Río
Paseo Bajo de la Virgen del Puerto, 28005, Madrid
915880432
DISTRITO CENTRO
----------
Cubo Gym Retiro
Pª de Fernán Núñez, (en "Capricho Castillete"), 28009, Madrid
915880432
DISTRITO RETIRO
----------
